In [125]:
#CSCE 5222 Feature Engineering
#Stock Market Price Predicition

#importing libraries
import numpy as np
import pandas as pd
import yfinance as yf
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import seaborn as sns


yf.pdr_override() #yfinance is our dataset
dataset = yf

#oneMonth = yf.download('COST', '2023-10-1', '2023-11-1') #downloads Costco stock price in October of this year

oct = yf.download('COST', '2013-1-1', '2023-11-1') #downloads Costco stock price from January 2013 to October of this year

#print(oneMonth) #displays the stock prices
#oneMonth = pd.DataFrame(oneMonth)
octdf = pd.DataFrame(oct) #creates dataframe to hold the data for october
octdf.head() #displays first five columns of the dataframe


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,100.599998,101.449997,100.209999,101.449997,82.717537,3153800
2013-01-03,102.110001,103.019997,101.760002,102.489998,83.565491,3872400
2013-01-04,102.550003,102.910004,101.550003,102.160004,83.296448,1989000
2013-01-07,101.089996,101.730003,100.900002,101.370003,82.652313,1663900
2013-01-08,101.000000,101.790001,100.730003,101.180000,82.497391,2189900


In [129]:
percentile75 = octdf.Close.quantile(0.75) #obtains the 75th percentile value of the closing price
percentile25 = octdf.Close.quantile(0.25) #obtains the 25th percentile value of the closing price

#equation for added feature is (daily close price - percentile25)/(percentile75 - percentile 25)
dailyValueProportion = [] #list to become part of the feature set
highlowDifference = [] #list to become part of the feature set
opencloseDifference = [] #list to become part of the feature set

for i in range (0, len(octdf) ): #loop that takes in all the rows of closing values and calculates the Daily Value Proportion feature from them
  dailyClosePrice = octdf.iloc[i].Close
  calculation = (dailyClosePrice - percentile25)/(percentile75 - percentile25) #
  dailyValueProportion.append(calculation)

for i in range (0, len(octdf) ): #obtains the high - low daily price
  highPrice = octdf.iloc[i].High
  lowPrice = octdf.iloc[i].Low
  calculation = (highPrice - lowPrice)
  highlowDifference.append(calculation)

for i in range (0, len(octdf) ): #obtains the close - open daily price
  openPrice = octdf.iloc[i].Close
  closePrice = octdf.iloc[i].Open
  calculation = (openPrice - closePrice)
  opencloseDifference.append(calculation)

octdf['Daily_Value_Proportion'] = dailyValueProportion #adds the Daily Value Proportion to the dataframe as a feature
octdf['H-L'] = highlowDifference #adds the difference between the highest and lowest price for the day as a feature to our feature set
octdf['O-C'] = opencloseDifference #adds the difference between the opening and closing price for the day as a feature to our feature set
print(octdf)



                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2013-01-02  100.599998  101.449997  100.209999  101.449997   82.717537   
2013-01-03  102.110001  103.019997  101.760002  102.489998   83.565491   
2013-01-04  102.550003  102.910004  101.550003  102.160004   83.296448   
2013-01-07  101.089996  101.730003  100.900002  101.370003   82.652313   
2013-01-08  101.000000  101.790001  100.730003  101.180000   82.497391   
...                ...         ...         ...         ...         ...   
2023-10-25  548.549988  553.830017  545.609985  549.989990  548.982483   
2023-10-26  549.650024  554.659973  545.530029  547.599976  546.596863   
2023-10-27  547.599976  548.030029  540.229980  543.030029  542.035278   
2023-10-30  545.739990  556.359985  543.640015  554.880005  553.863525   
2023-10-31  552.159973  554.030029  549.059998  552.440002  551.427979   

             Volume  Daily_Value_Prop

In [130]:
#dropping adjusted closing price from our dataset
newdf = octdf.drop(columns = ('Adj Close'), axis = 1)
#removes volume as a feature from our dataset
newdf = newdf.drop(columns = ('Volume'), axis = 1 )
print(newdf)

X = newdf.drop('Close', axis = 1) #close is the target value and needs to be dropped
y = newdf.Close #y is the target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 43459, shuffle = True) #uses a 20% train/test split

regressionModel = KNeighborsRegressor(n_neighbors = 4) #uses 4 neighbors for regression
regressionModel.fit(X_train, y_train) #fits the model, then makes predictions
modelPredictions = regressionModel.predict(X_test)
result = r2_score(y_test, modelPredictions) #obtain a result

print("Using 4 neigbors, the r2score result is: " ) #prints results
print(result)



                  Open        High         Low       Close  \
Date                                                         
2013-01-02  100.599998  101.449997  100.209999  101.449997   
2013-01-03  102.110001  103.019997  101.760002  102.489998   
2013-01-04  102.550003  102.910004  101.550003  102.160004   
2013-01-07  101.089996  101.730003  100.900002  101.370003   
2013-01-08  101.000000  101.790001  100.730003  101.180000   
...                ...         ...         ...         ...   
2023-10-25  548.549988  553.830017  545.609985  549.989990   
2023-10-26  549.650024  554.659973  545.530029  547.599976   
2023-10-27  547.599976  548.030029  540.229980  543.030029   
2023-10-30  545.739990  556.359985  543.640015  554.880005   
2023-10-31  552.159973  554.030029  549.059998  552.440002   

            Daily_Value_Proportion        H-L       O-C  
Date                                                     
2013-01-02               -0.207033   1.239998  0.849998  
2013-01-03         